# Computer Vision 1: Assignment 1

Group Number 43

Marek Drwal

## setup

In [18]:
import imageio
import numpy as np
import skimage as ski

## task 1

Learning Multiple Layers of Features from Tiny Images, Alex Krizhevsky, 2009, https://www.cs.toronto.edu/~kriz/learning-features-2009-TR.pdf

In [19]:
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

In [20]:
data_dir = "C:/Users/admin/Documents/cifar-10-python/cifar-10-batches-py"

In [21]:
data_batch_1 = unpickle(data_dir + '/data_batch_1')

In [22]:
label_names = unpickle(data_dir + '/batches.meta')

In [23]:
# extracted_classes = data_batch_1[b'data'][data_batch_1[b'labels'] in label_names[b'label_names'][1, 4, 8]]
extracted_classes = data_batch_1[b'data'][data_batch_1[b'labels'] in [1,4,8]]

In [24]:
data_batch_1[b'data'][np.isin(np.array(data_batch_1[b'labels']), [1,4,8])]

array([[ 28,  37,  38, ...,  28,  37,  46],
       [170, 168, 177, ...,  82,  78,  80],
       [159, 150, 153, ...,  14,  17,  19],
       ...,
       [116, 120, 126, ...,  82,  84,  81],
       [ 71,  60,  74, ...,  68,  69,  68],
       [250, 254, 211, ..., 215, 255, 254]], dtype=uint8)

In [25]:
data_batch_1_labels = np.array(data_batch_1[b'labels'])
extracted_top_30_list = []

for class_lab in [1,4,8]:
    first30 = data_batch_1[b'data'][data_batch_1_labels == class_lab][:30,]
    print(len(first30))
    extracted_top_30_list.append(first30)

extracted_top_30_list

30
30
30


[array([[170, 168, 177, ...,  82,  78,  80],
        [159, 150, 153, ...,  14,  17,  19],
        [ 50,  51,  42, ..., 166, 166, 162],
        ...,
        [214, 212, 214, ...,  58,  60,  61],
        [ 48, 128, 156, ...,  20,  19,  30],
        [ 47,  57,  70, ...,   2,   2,   3]], dtype=uint8),
 array([[ 28,  37,  38, ...,  28,  37,  46],
        [ 53,  54,  56, ...,  39,  46,  41],
        [153, 174, 155, ..., 217, 217, 219],
        ...,
        [ 45,  51,  77, ..., 116, 113, 134],
        [ 30,  27,  29, ...,  64,  62,  67],
        [ 89,  85,  81, ...,  75,  60,  48]], dtype=uint8),
 array([[134, 131, 128, ..., 136, 137, 138],
        [ 76,  73,  69, ..., 127, 127, 130],
        [151, 150, 151, ..., 102, 102, 103],
        ...,
        [ 58,  51,  50, ...,  70,  63,  57],
        [196, 203, 212, ...,  94, 100, 106],
        [ 61,  62,  62, ...,  92,  90,  91]], dtype=uint8)]

In [26]:
extracted_top_30_list

[array([[170, 168, 177, ...,  82,  78,  80],
        [159, 150, 153, ...,  14,  17,  19],
        [ 50,  51,  42, ..., 166, 166, 162],
        ...,
        [214, 212, 214, ...,  58,  60,  61],
        [ 48, 128, 156, ...,  20,  19,  30],
        [ 47,  57,  70, ...,   2,   2,   3]], dtype=uint8),
 array([[ 28,  37,  38, ...,  28,  37,  46],
        [ 53,  54,  56, ...,  39,  46,  41],
        [153, 174, 155, ..., 217, 217, 219],
        ...,
        [ 45,  51,  77, ..., 116, 113, 134],
        [ 30,  27,  29, ...,  64,  62,  67],
        [ 89,  85,  81, ...,  75,  60,  48]], dtype=uint8),
 array([[134, 131, 128, ..., 136, 137, 138],
        [ 76,  73,  69, ..., 127, 127, 130],
        [151, 150, 151, ..., 102, 102, 103],
        ...,
        [ 58,  51,  50, ...,  70,  63,  57],
        [196, 203, 212, ...,  94, 100, 106],
        [ 61,  62,  62, ...,  92,  90,  91]], dtype=uint8)]

In [27]:
len(extracted_top_30_list)

3